In [1]:
import pandas as pd
import numpy as np
import dataretrieval.nwis as nwis
from datetime import date
from calendar import month_name

from bokeh.models import Band, ColumnDataSource, LabelSet
from bokeh.plotting import figure, show
from bokeh.models import DatetimeTickFormatter
from datetime import datetime, timedelta

bokehoutfile = '/Users/greg/Documents/python/usgs/dallas_creek.html'

SITES = ['09147000', '09146200', '09146020', '09147025', '09147500']
df = nwis.get_record(sites=SITES, service='site')
SITE_NOS = df.site_no.to_list()
SITE_NAMES = df.station_nm.to_list()
df.station_nm

0                 UNCOMPAHGRE RIVER NEAR OURAY, CO
1              UNCOMPAHGRE RIVER NEAR RIDGWAY, CO.
2                    DALLAS CREEK NEAR RIDGWAY, CO
3    UNCOMPAHGRE RIVER BELOW RIDGWAY RESERVOIR, CO
4                  UNCOMPAHGRE RIVER AT COLONA, CO
Name: station_nm, dtype: object

In [2]:
site = 2 #dallas creek
site_name = SITE_NAMES[site]
start_date = '2000-01-01'
end_date = date.today()

df = nwis.get_record(sites=SITE_NOS[site], service='iv', start=start_date, end=end_date)
print(f'Instantaneous records for {SITE_NAMES[site]}')
print(f'Number of records: {df.shape[0]}')
print(f'Earliest date: {df.index.min():%B %d, %Y, %H:%M}')
print(f'Latest date: {df.index.max():%B %d, %Y, %H:%M}')
print('\ndataframe info:')
print(df.info())

df['discharge'] = df['00060']  # discharge in ft3 per second
df.reset_index(inplace=True)
df['water_year'] = df.datetime.dt.year + (df.datetime.dt.month > 9)

nwisdf = df
nwisdf.sample(5).sort_values('datetime')

Instantaneous records for DALLAS CREEK NEAR RIDGWAY, CO
Number of records: 680018
Earliest date: January 10, 2000, 07:00
Latest date: June 14, 2024, 21:00

dataframe info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 680018 entries, 2000-01-10 07:00:00+00:00 to 2024-06-14 21:00:00+00:00
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   site_no   680018 non-null  object 
 1   00045     213382 non-null  float64
 2   00045_cd  213382 non-null  object 
 3   00060     668187 non-null  float64
 4   00060_cd  668187 non-null  object 
 5   00065     106487 non-null  float64
 6   00065_cd  106487 non-null  object 
dtypes: float64(3), object(4)
memory usage: 41.5+ MB
None


,datetime,site_no,00045,00045_cd,00060,00060_cd,00065,00065_cd,discharge,water_year
304593,2010-12-26 00:30:00+00:00,09147000,NaN,NaN,22.00,A,NaN,NaN,22.00,2011
422066,2015-05-15 00:45:00+00:00,09147000,0.0,A,10.00,A,NaN,NaN,10.00,2015
509852,2018-08-11 08:15:00+00:00,09147000,NaN,NaN,0.59,A,NaN,NaN,0.59,2018
546464,2019-12-29 12:30:00+00:00,09147000,NaN,NaN,17.80,"A, e",NaN,NaN,17.80,2020
639321,2023-03-28 12:15:00+00:00,09147000,NaN,NaN,NaN,NaN,2.45,A,NaN,2023


The following creates a dataset in which all of the data from 2000 on is collapes into a single year, which ends at the end of the final (most recent) day in the dataset.  If the most recent date, for example, is June 3 and the dataset contains 20 years of data, there will be 20 plots, each with a different "display_year" running from June 4 the previous year to June 3.  For display_year=2023, for example, the plot would go from June 4, 2022, to June 3, 2023.  These can then all be overlapped on one plot (in this case, going from June 4 to June 3 the next year).  The year is suppressed on axis of hte graph to avoid confusion, but hte datetime, which is preserved from the original, is available, and shows up by means of the hover tool.

In [3]:
# make copy of dataframe for plotting
df = nwisdf[['datetime','discharge', 'water_year']].copy()
df['date_string'] = df.datetime.dt.strftime('%Y-%m-%d %H:%M:%S')
max_date = df.datetime.max()
max_year = max_date.year
def calc_display_date(row): # takes datetime object, returns (display_year, display_date)
    d = row.datetime
    y = d.year
    d = d.replace(year=2024) # changes all years to an arbitrary leap year for display purposes
    if d > max_date.replace(year=2024):  
        y += 1
        d = d.replace(year = 2023)
    return y, d
df[['display_year', 'display_date']] = df.apply(calc_display_date, axis=1, result_type='expand')

# determine initial x-axis start and end points for plot
initial_end_date = max_date.replace(year=2024) + timedelta(days=1)
initial_start_date = initial_end_date - timedelta(days=31)

# determine y-axis limits for graph: min is set to df minimum (but not 0) and max to 95th percentile
y_axis_min = df[df.discharge>0].discharge.min()
y_axis_max = df.discharge.quantile(0.95)

cpast = 'lightseagreen' #color of past years' plots
ccurrent = 'firebrick' #color of current year's plots
cquantile = 'black' #color of quantile bands and median

p = figure(tools='hover,xpan,xzoom_in,xzoom_out,xwheel_zoom,reset,save', 
           toolbar_location='right', 
           title=f'Instant discharge at {site_name} since {df.datetime.min():%B %Y}', 
           x_axis_type='datetime',
           y_axis_type='log',
           x_range=(initial_start_date, initial_end_date),
           y_range=(y_axis_min, y_axis_max),
           height=800,
           width=1200)
p.toolbar.logo = "grey"
p.background_fill_color = "#efefef"
p.xaxis.axis_label = "date"
p.xaxis.formatter = DatetimeTickFormatter(days='%m/%d', months='%m/%d', years='%m/%d') # supprresses display of year
p.yaxis.axis_label = "discharge (cu ft per sec)"
p.grid.grid_line_color = "white"
p.hover.tooltips = [
    ("date", "@date_string"),
    ("discharge:", "@discharge{0.00} cu ft / sec")
]

for i, group in df.groupby('display_year'):
    source = ColumnDataSource(group)
    if i == max_year:
        line_width = 3
        line_color = ccurrent  # uses css color names
    else:
        line_width = 1
        line_color = cpast
    p.line('display_date','discharge', source=source, line_width=line_width, line_color=line_color) # legend_group='water_year'

In [4]:
df['date'] = pd.to_datetime(df.display_date.dt.date)
df.date = df.date + timedelta(hours=12)
quantdf = pd.DataFrame()
for pct in range(10, 100, 10):
    df2 = df.groupby('date').discharge.quantile(pct/100).rename(f'pctile{pct}')
    quantdf = quantdf.join(df2, how='outer')
quantdf.reset_index(inplace=True)
quantdf.rename(columns={'date':'display_date', 'pctile50':'median'}, inplace=True)
source = ColumnDataSource(quantdf)
p.line('display_date','median', source=source, line_width=3, line_color=cquantile, alpha=0.3)
for pct in range(40, 0, -10):
    band = Band(base='display_date', lower=f'pctile{pct}', upper=f'pctile{100-pct}', source=source,
                fill_alpha=0.1, fill_color=cquantile)
    p.add_layout(band)
show(p)

In [5]:
from bokeh.resources import CDN
from bokeh.embed import file_html

html = file_html(p, CDN, 'dallas creek')
f = open(bokehoutfile, 'a')
f.write(html)
f.close()